## 14 Error Handling
* 14.1 try/except Structures
* 14.1.1 Using Named Exceptions
* 14.1.2 Multiple except Blocks
* 14.1.3 Error Handling Gotcha
* 14.2 Geoprocessing and Error Handling
* 14.2.1 Getting Geoprocessing Messages
* 14.2.2 The arcpy Named Exception
* 14.3 Catching Exceptions in Loops
* 14.4 Discussion
* 14.5 Key Terms
* 14.6 Exercises

In [ ]:
# %load script/multRasts.py
# multRast.py
# Purpose: Multiply each raster in a database by an input factor.
# Usage: numeric_value
# Example input: 5
import arcpy, sys

# Set multiplication factor
factor = float(sys.argv[1])

arcpy.env.overwriteOutput = True
arcpy.env.worspace = 'C:/gispy/data/ch14/rastTester.gdb'
arcpy.CheckOutExtension('Spatial')
outDir = 'C:/gispy/scratch/'

# Get raster list & multiply each by a factor
rasterList = arcpy.ListRasters()
for rasterImage in rasterList:
    rasterObj = arcpy.Raster(rasterImage)
    rastMult = rasterObj * factor
    rastMult.save(outDir + rasterImage)
    del rastMult


In [11]:
%run scriptPt/multRasts.py 5

## 14.1 try/except Structures

In [ ]:
# %load script/doubleMyNumber.py
# doubleMyNumber.py
# Purpose: Multiply input by 2.
# Usage: numeric_value
# Example input: 5
import sys

try:
    number = float(sys.argv[1])
    product = 2*number
    print 'The doubled number is {0}'.format(product)
except:
    print 'An error occurred.'
    print 'Please enter a numerical argument.'

print 'Good bye!'


In [13]:
%run scriptPt/doubleMyNumber.py 5

The doubled number is 10.0
Good bye!


## 14.1.1 Using Named Exceptions

In [ ]:
# %load script/doubleMyNumberV2.py
# doubleMyNumberV2.py
# Purpose: Multiply input by 2
#          or catch value error.
# Usage: numeric_value
# Example input: 5
import sys

try:
    number = float(sys.argv[1])
    product = 2*number
    print 'The doubled number is {0}'.format(product)
except ValueError:
    print 'Input must be numerical.'
print 'Good bye!'


In [15]:
%run scriptPt/doubleMyNumberV2.py 5

The doubled number is 10.0
Good bye!


## 14.1.2 Multiple except Blocks

In [ ]:
# %load script/slopeTry.py
# slopeTry.py
# Purpose: Find the slope, given rise and run.
# Usage: slope_rise slope_run
# Example input: 1 3
import sys

rise = sys.argv[1]
run = sys.argv[2]

try:
    print 'Rise: {0} Run: {1}'.format(rise, run)
    slope = float(rise)/float(run)
    print 'Slope = rise/run'
except ZeroDivisionError:
    slope = 'Undefined (line is vertical)'
except ValueError:
    print 'Usage: <numeric rise> <numeric run>'
    slope = 'Not found'

print 'Slope:', slope


In [17]:
%run scriptPt/slopeTry.py 1 3

Rise: 1 Run: 3
Slope = rise/run
Slope: 0.333333333333


## 14.1.3 Error Handling Gotcha

In [ ]:
# %load script/cubeMyNumber.py
# cubeMyNumber.py
# Purpose: Cube input value.
# Usage: numeric_value
# Example input: 5
import sys
try:
    number = float(sys.argv[1])
    cube = number**3
    print 'The cubed number is {0}'.format()  # missing arg
except:
    print 'Input must be numerical.'
print 'Good bye!'


In [19]:
%run scriptPt/cubeMyNumber.py 5

Input must be numerical.
Good bye!


In [ ]:
# %load script/cubeMyNumberV2.py
# cubeMyNumberV2.py
# Purpose: Cube input value
#   or print traceback message.
# Usage: numeric_value
# Example input: 5
import sys, traceback
try:
    number = float(sys.argv[1])
    cube = number**3
    print 'The cubed number is {0}'.format()  # missing arg
except:
    print 'Input must be numerical.'
    traceback.print_exc()
print 'Good bye!'


In [22]:
%run script/cubeMyNumberV2.py 5

Input must be numerical.
Good bye!


Traceback (most recent call last):
  File "D:\BOOKS\GISen\_PYTHON\PythonForArcGIS\SF_PFA\ch14\script\cubeMyNumberV2.py", line 10, in <module>
    print 'The cubed number is {0}'.format()  # missing arg
IndexError: tuple index out of range


## 14.2 Geoprocessing and Error Handling

## 14.2.1 Getting Geoprocessing Messages

In [23]:
arcpy.GetMessages()

''

---

In [26]:
inputFile = 'data/cover.shp'
count = arcpy.GetCount_management(inputFile)
print arcpy.GetMessages()

Executing: GetCount data/cover.shp
Start Time: Mon Apr 25 23:03:57 2016
Row Count = 426
Succeeded at Mon Apr 25 23:03:57 2016 (Elapsed Time: 0.00 seconds)


---

In [27]:
inputFile = 'data/parkLines.shp'
outputFile = 'scratch/buffer.shp'
arcpy.Buffer_analysis(inputFile, outputFile, '1 mile')

<Result 'scratch\\buffer.shp'>

In [29]:
print arcpy.GetMessages()

Executing: Buffer data/parkLines.shp scratch\buffer.shp "1 Miles" FULL ROUND NONE #
Start Time: Mon Apr 25 23:04:31 2016
Succeeded at Mon Apr 25 23:04:32 2016 (Elapsed Time: 1.00 seconds)


In [30]:
print arcpy.GetMessage(0)

Executing: Buffer data/parkLines.shp scratch\buffer.shp "1 Miles" FULL ROUND NONE #


In [31]:
arcpy.GetMessageCount()

3

In [32]:
print arcpy.GetMessage(arcpy.GetMessageCount() - 1)

Succeeded at Mon Apr 25 23:04:32 2016 (Elapsed Time: 1.00 seconds)


In [34]:
count = arcpy.GetCount_management()

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000735: Input Rows: Value is required
Failed to execute (GetCount).


In [35]:
print arcpy.GetMessages()

Executing: GetCount #
Start Time: Mon Apr 25 23:05:25 2016
Failed to execute. Parameters are not valid.
ERROR 000735: Input Rows: Value is required
Failed to execute (GetCount).
Failed at Mon Apr 25 23:05:25 2016 (Elapsed Time: 0.00 seconds)


## 14.2.2 The arcpy Named Exception

In [ ]:
# %load script/bufferTry.py
# bufferTry.py
# Purpose: Buffer the input dataset.
# Usage: Fullpath_filename outDir
# Example input: C:/gispy/data/ch14/parkLines.shp outputWorkspace
import arcpy, sys, os
arcpy.env.overwriteOutput = True
try:
    inFile = sys.argv[1]
    outDir = sys.argv[2]
    buffer = outDir + '/' + os.path.splitext(inFile)[0] + 'Buff.shp'
    arcpy.Buffer_analysis(inFile, buffer, '1 mile')
    print '{0} created.'.format(buffer)
except arcpy.ExecuteError:
    print arcpy.GetMessages()
except IndexError:
    print 'Usage: <full path shapefile name>'


In [41]:
%run script/bufferTry.py data/parkLines.shp scratch/

Executing: Buffer data/parkLines.shp scratch//data/parkLinesBuff.shp "1 Miles" FULL ROUND NONE #
Start Time: Mon Apr 25 23:09:25 2016
ERROR 000210: Cannot create output scratch//data/parkLinesBuff.shp
Failed to execute (Buffer).
Failed at Mon Apr 25 23:09:25 2016 (Elapsed Time: 0.00 seconds)


## 14.3 Catching Exceptions in Loops

In [ ]:
# %load script/buffeLoopTry.py
# bufferLoopTry.py
# Purpose: Buffer the feature classes in a workspace.
# Usage: No arguments needed.
import arcpy, os
arcpy.env.overwriteOutput = True
arcpy.env.workspace = 'C:/gispy/data/ch14'
outDir = 'C:/gispy/scratch/'
fcs = arcpy.ListFeatureClasses()
distance = '500 meters'

for fc in fcs:
    outFile = outDir + os.path.splitext(fc)[0] + 'Buff.shp'
    try:
        arcpy.Buffer_analysis(fc, outFile, distance)
        print 'Created: {0}'.format(outFile)
    except arcpy.ExecuteError:
        print arcpy.GetMessage(2)


In [38]:
%run scriptPt/buffeLoopTry.py

Created: scratch/bufferBuff.shp
Created: scratch/coverBuff.shp
ERROR 000229: Cannot open data\dummyFile.shp
Created: scratch/firesBuff.shp
Created: scratch/no_damageBuff.shp
Created: scratch/parkLinesBuff.shp


In [ ]:
# %load script/bufferLoopDistTry.py
# bufferLoopDistTry.py
# Purpose: Buffer the input file by the given distance.
# Usage: input_filename numeric_distance (using the default unit of measure)
# Example input: C:/gispy/data/ch14/cover.shp 3

import arcpy, sys, os
arcpy.env.workspace = os.path.dirname(sys.argv[1])
fc = os.path.basename(sys.argv[1])
outDir = 'C:/gispy/scratch/'
arcpy.env.overwriteOutput = False
maxDist = float(sys.argv[2])
i = 1
while i <= maxDist:
    try:
        outFile = outDir + os.path.splitext(fc)[0] + str(i) + 'Buff.shp'
        distance = str(i) + ' miles'
        arcpy.Buffer_analysis(fc, outFile, distance)
        print 'Created: ', outFile
    except arcpy.ExecuteError:
        print arcpy.GetMessage(3)
    i = i + 1


In [3]:
%run scriptPt/bufferLoopDistTry.py data/cover.shp 3

Created:  scratch/cover1Buff.shp
Created:  scratch/cover2Buff.shp
Created:  scratch/cover3Buff.shp


## 14.4 Discussion

## 14.5 Key Terms

## 14.6 Exercises